In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
from knapsack_problem.knapsack import best_full_teams
from knapsack_problem.player import Player
import pandas as pd
import math
from config import *

In [3]:
DATA_PATH = "predictions/preds_next_season.csv"

In [4]:
df = pd.read_csv(DATA_PATH).rename(columns={"Pred": "Prediction"})

In [5]:
df.columns

Index(['Name_original', 'GW', 'Season', 'Team', 'Opponent', 'Was_home',
       'Team_rating', 'Opp_rating', 'Price', 'FPL_pos', 'FPL_points',
       'Prediction'],
      dtype='object')

In [6]:
df = df[df["GW"] == NEXT_GAMEWEEK]

In [7]:
df.head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction
15,James Maddison,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,7.8,MID,NaN,7.521450
47,Erling Haaland,5.0,2023-24,Manchester City,West Ham,0.0,2087.353760,1807.522339,14.1,FWD,NaN,7.093404
98,Mohamed Salah,5.0,2023-24,Liverpool,Wolves,0.0,1961.337402,1708.177246,12.5,MID,NaN,6.706296
121,Destiny Udogie,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,4.7,DEF,NaN,6.446187
124,James Ward-Prowse,5.0,2023-24,West Ham,Manchester City,1.0,1807.522339,2087.353760,6.1,MID,NaN,6.425669
162,Bruno Borges Fernandes,5.0,2023-24,Manchester Utd,Brighton,1.0,1860.949707,1839.788574,8.4,MID,NaN,6.118914
215,Guglielmo Vicario,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,5.1,GK,NaN,5.630467
248,Moussa Diaby,5.0,2023-24,Aston Villa,Crystal Palace,1.0,1828.641479,1768.190308,6.6,MID,NaN,5.520411
264,Bukayo Saka,5.0,2023-24,Arsenal,Everton,0.0,1928.752808,1682.138794,8.7,MID,NaN,5.445979
319,Jarrod Bowen,5.0,2023-24,West Ham,Manchester City,1.0,1807.522339,2087.353760,7.1,MID,NaN,5.276847


In [8]:
# df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [9]:
df.shape, df[df["FPL_pos"] == "GK"].shape, df[df["FPL_pos"] == "DEF"].shape, df[df["FPL_pos"] == "MID"].shape, df[df["FPL_pos"] == "FWD"].shape

((537, 12), (59, 12), (189, 12), (226, 12), (63, 12))

In [10]:
df.head()

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction
15,James Maddison,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,7.8,MID,NaN,7.521450
47,Erling Haaland,5.0,2023-24,Manchester City,West Ham,0.0,2087.353760,1807.522339,14.1,FWD,NaN,7.093404
98,Mohamed Salah,5.0,2023-24,Liverpool,Wolves,0.0,1961.337402,1708.177246,12.5,MID,NaN,6.706296
121,Destiny Udogie,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,4.7,DEF,NaN,6.446187
124,James Ward-Prowse,5.0,2023-24,West Ham,Manchester City,1.0,1807.522339,2087.353760,6.1,MID,NaN,6.425669


In [11]:
playersDB = []
for i, row in df.iterrows():
    playersDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [12]:
df_original = df.copy()

In [13]:
df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [14]:
smallDB = []
for i, row in df.iterrows():
    smallDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [15]:
def find_most_expensive(playersDB):
    index = 0
    price = 0
    for i, p in enumerate(playersDB):
        if p.price > price:
            index = i
            price = p.price
    return index, price
    

In [16]:
def find_N_cheapest_on_position(playersDB, n_players, position):
    output = []
    for p in playersDB:
        if p.position == position:
            if len(output) < n_players:
                output.append(p)
            else:
                i, price = find_most_expensive(output)
                if p.price < price:
                    output[i] = p
    return output

In [17]:
find_N_cheapest_on_position(playersDB, 1, "FWD")[0].__str__()

'(Shola Shoretire, FWD, 4.4, 0.62, Manchester Utd) - (fixtures: 1.0)'

In [18]:
# for p in playersDB:
#     print(p.__str__())

In [19]:
# 2 example Goalkeepers
# playersDB.append(Player("GK1", "GK", 5, 4, "Manchester Utd"))
# playersDB.append(Player("GK2", "GK", 4, 3, "Nott'ham Forest"))

In [20]:
possible_formations = [
    [3, 4, 3],
    [3, 5, 2],
    [4, 3, 3],
    [4, 4, 2],
    [4, 5, 1],
    [5, 2, 3],
    [5, 3, 2],
    [5, 4, 1],
]

In [21]:
budget = 100

In [22]:
def fill_bench(playersDB, formation):
    n_gk, n_def, n_mid, n_fwd = 1, 5 - formation[0], 5 - formation[1], 3 - formation[2]
    gks = find_N_cheapest_on_position(playersDB, n_gk, "GK")
    defs = find_N_cheapest_on_position(playersDB, n_def, "DEF")
    mids = find_N_cheapest_on_position(playersDB, n_mid, "MID")
    fwds = find_N_cheapest_on_position(playersDB, n_fwd, "FWD")
    print(formation, n_gk, n_def, n_mid, n_fwd)
    bench = gks + defs + mids + fwds
    for p in bench:
        print(p.__str__())
        p.benched = True
    bench_price = sum([p.price for p in bench])
    print(bench_price)
    return bench, bench_price

In [23]:
best_teams = []
best_team = None
best_team_score = 0
best_bench = None

for formation in possible_formations:
    bench, bench_price = fill_bench(playersDB, formation)
    best_team_for_formation = best_full_teams(smallDB, [formation], budget - bench_price)
    
    if best_team_for_formation[0][1] > best_team_score:
        best_team = best_team_for_formation
        best_team_score = best_team_for_formation[0][1]
        best_bench = bench
        
for p in best_team[0][2]:
    p.benched = True
for p in best_bench:
    best_team[0][2].append(p)
        
    

[3, 4, 3] 1 2 1 0
(Matt Turner, GK, 4.0, 0.79, Nott'ham Forest) - (fixtures: 1.0)
(Tariq Lamptey, DEF, 3.9, 1.67, Brighton) - (fixtures: 1.0)
(Sergio Gómez, DEF, 3.9, 0.7, Manchester City) - (fixtures: 1.0)
(Isaac Hayden, MID, 4.4, 1.66, Newcastle Utd) - (fixtures: 1.0)
16.200000000000003
With formation [3, 4, 3]: 65.12  | (price = 81.8)
(Guglielmo Vicario, GK, 5.1, 5.63, Tottenham) - (fixtures: 1.0)
(Destiny Udogie, DEF, 4.7, 6.45, Tottenham) - (fixtures: 1.0)
(Axel Disasi, DEF, 5.1, 4.8, Chelsea) - (fixtures: 1.0)
(Micky van de Ven, DEF, 4.5, 4.38, Tottenham) - (fixtures: 1.0)
(James Maddison, MID, 7.8, 7.52, Tottenham) - (fixtures: 1.0)
(Mohamed Salah, MID, 12.5, 6.71, Liverpool) - (fixtures: 1.0)
(James Ward-Prowse, MID, 6.1, 6.43, West Ham) - (fixtures: 1.0)
(Bruno Borges Fernandes, MID, 8.4, 6.12, Manchester Utd) - (fixtures: 1.0)
(Erling Haaland, FWD, 14.1, 7.09, Manchester City) - (fixtures: 1.0)
(Ollie Watkins, FWD, 8.0, 5.01, Aston Villa) - (fixtures: 1.0)
(Carlton Morris, FW

In [24]:
for p in best_team[0][2]:
    print(p.__str__())

(Guglielmo Vicario, GK, 5.1, 5.63, Tottenham) - (fixtures: 1.0)
(Destiny Udogie, DEF, 4.7, 6.45, Tottenham) - (fixtures: 1.0)
(Axel Disasi, DEF, 5.1, 4.8, Chelsea) - (fixtures: 1.0)
(Micky van de Ven, DEF, 4.5, 4.38, Tottenham) - (fixtures: 1.0)
(James Maddison, MID, 7.8, 7.52, Tottenham) - (fixtures: 1.0)
(Mohamed Salah, MID, 12.5, 6.71, Liverpool) - (fixtures: 1.0)
(James Ward-Prowse, MID, 6.1, 6.43, West Ham) - (fixtures: 1.0)
(Bruno Borges Fernandes, MID, 8.4, 6.12, Manchester Utd) - (fixtures: 1.0)
(Moussa Diaby, MID, 6.6, 5.52, Aston Villa) - (fixtures: 1.0)
(Erling Haaland, FWD, 14.1, 7.09, Manchester City) - (fixtures: 1.0)
(Ollie Watkins, FWD, 8.0, 5.01, Aston Villa) - (fixtures: 1.0)
(Matt Turner, GK, 4.0, 0.79, Nott'ham Forest) - (fixtures: 1.0)
(Tariq Lamptey, DEF, 3.9, 1.67, Brighton) - (fixtures: 1.0)
(Sergio Gómez, DEF, 3.9, 0.7, Manchester City) - (fixtures: 1.0)
(Shola Shoretire, FWD, 4.4, 0.62, Manchester Utd) - (fixtures: 1.0)


In [25]:
%%time
# best_team = best_full_teams(smallDB, possible_formations, budget)

CPU times: total: 0 ns
Wall time: 0 ns


In [26]:
f"Best team score: {best_team[0][1]}"

'Best team score: 65.66'

In [27]:
best_team[0]

([3, 5, 2],
 65.66,
  <knapsack_problem.player.Player at 0x2296ede3dc0>])

In [28]:
def best_team_to_df(best_team_squad, df):
    names = [p.name for p in best_team_squad]
    print(names)
    custom_fpl_order = ["FWD", "MID", "DEF", "GK"]
    
    # Convert "FPL_pos" column to categorical with custom order
    df["FPL_pos"] = pd.Categorical(df["FPL_pos"], categories=custom_fpl_order, ordered=True)
    
    # Filter by names and sort by custom ordered "FPL_pos" and "Price"
    output = df[df["Name_original"].isin(names)].reset_index()
    output["First_eleven"] = True
    output.loc[11:15, "First_eleven"] = False
    output = output.sort_values(by=["First_eleven", "FPL_pos", "Price"], ascending=False)
    
    return output

In [29]:
best_squad = best_team_to_df(best_team[0][2], df_original)
best_squad

['Guglielmo Vicario', 'Destiny Udogie', 'Axel Disasi', 'Micky van de Ven', 'James Maddison', 'Mohamed Salah', 'James Ward-Prowse', 'Bruno Borges Fernandes', 'Moussa Diaby', 'Erling Haaland', 'Ollie Watkins', 'Matt Turner', 'Tariq Lamptey', 'Sergio Gómez', 'Shola Shoretire']


,index,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction,First_eleven
6,215,Guglielmo Vicario,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,5.1,GK,NaN,5.630467,True
9,494,Axel Disasi,5.0,2023-24,Chelsea,Bournemouth,0.0,1774.226440,1659.008667,5.1,DEF,NaN,4.795500,True
3,121,Destiny Udogie,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,4.7,DEF,NaN,6.446187,True
10,853,Micky van de Ven,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,4.5,DEF,NaN,4.375095,True
2,98,Mohamed Salah,5.0,2023-24,Liverpool,Wolves,0.0,1961.337402,1708.177246,12.5,MID,NaN,6.706296,True
5,162,Bruno Borges Fernandes,5.0,2023-24,Manchester Utd,Brighton,1.0,1860.949707,1839.788574,8.4,MID,NaN,6.118914,True
0,15,James Maddison,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,7.8,MID,NaN,7.521450,True
7,248,Moussa Diaby,5.0,2023-24,Aston Villa,Crystal Palace,1.0,1828.641479,1768.190308,6.6,MID,NaN,5.520411,True
4,124,James Ward-Prowse,5.0,2023-24,West Ham,Manchester City,1.0,1807.522339,2087.353760,6.1,MID,NaN,6.425669,True
1,47,Erling Haaland,5.0,2023-24,Manchester City,West Ham,0.0,2087.353760,1807.522339,14.1,FWD,NaN,7.093404,True


In [30]:
best_squad.to_csv("predictions/best_squad.csv")